In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
from dataset import load_svhn, random_split_train_val
from gradient_check import check_layer_gradient, check_layer_param_gradient, check_model_gradient
from layers import FullyConnectedLayer, ReLULayer
from model import TwoLayerNet
from trainer import Trainer, Dataset
from optim import SGD, MomentumSGD
from metrics import multiclass_accuracy

In [3]:
def prepare_for_neural_network(train_X, test_X):
    train_flat = train_X.reshape(train_X.shape[0], -1).astype(float) / 255.0
    test_flat = test_X.reshape(test_X.shape[0], -1).astype(float) / 255.0
    
    # Subtract mean
    mean_image = np.mean(train_flat, axis = 0)
    train_flat -= mean_image
    test_flat -= mean_image
    
    return train_flat, test_flat
    
train_X, train_y, test_X, test_y = load_svhn("data", max_train=10000, max_test=1000)    
train_X, test_X = prepare_for_neural_network(train_X, test_X)
# Split train into train and val
train_X, train_y, val_X, val_y = random_split_train_val(train_X, train_y, num_val = 1000)

Как всегда, начинаем с кирпичиков

Мы будем реализовывать необходимые нам слои по очереди. Каждый слой должен реализовать:

прямой проход (forward pass), который генерирует выход слоя по входу и запоминает необходимые данные
обратный проход (backward pass), который получает градиент по выходу слоя и вычисляет градиент по входу и по параметрам
Начнем с ReLU, у которого параметров нет.

In [16]:
# TODO: Implement ReLULayer layer in layers.py
# Note: you'll need to copy implementation of the gradient_check function from the previous assignment

X = np.array([[1,-2,3],
              [-1, 2, 0.1]])

assert check_layer_gradient(ReLULayer(), X)

CHECK GRADIENT
predictions is 
 [[ 1.  -2.   3. ]
 [-1.   2.   0.1]]
analytic grad is 
 [[ 0.32526653  0.          1.52832059]
 [-0.         -0.04901917  0.55115977]]
numeric grad array is 
 [[ 0.32526653  0.          1.52832059]
 [ 0.         -0.04901917  0.55115977]]
Gradient check passed!


In [45]:
# TODO: Implement FullyConnected layer forward and backward methods
assert check_layer_gradient(FullyConnectedLayer(3, 4), X)
# TODO: Implement storing gradients for W and B
assert check_layer_param_gradient(FullyConnectedLayer(3, 4), X, 'W')
assert check_layer_param_gradient(FullyConnectedLayer(3, 4), X, 'B')

CHECK GRADIENT
x is 
 [[ 1.  -2.   3. ]
 [-1.   2.   0.1]]
analytic grad is 
 [[ 0.          0.          0.        ]
 [ 0.00079777 -0.00032891  0.00108752]]
numeric grad array is 
 [[ 0.          0.          0.        ]
 [ 0.00079777 -0.00032891  0.00108752]]
Gradient check passed!
CHECK GRADIENT
x is 
 [[ 6.69858758e-04  6.21307957e-04 -1.90820258e-03  3.80195894e-04]
 [ 4.41898493e-04 -6.62399144e-04  1.66768421e-03  1.08766623e-03]
 [-1.83024383e-04 -5.95968305e-04  1.65144446e-03 -1.90137379e-05]]
analytic grad is 
 [[ 0.81073325 -0.83106266 -1.27102401 -0.15674634]
 [-1.62146651  1.66212533  2.54204803  0.31349268]
 [-0.08107333 -2.49318799 -1.08034587  0.01567463]]
numeric grad array is 
 [[ 0.81073325 -0.83106266 -1.27102401 -0.15674634]
 [-1.62146651  1.66212533  2.54204803  0.31349268]
 [-0.08107333 -2.49318799 -1.08034587  0.01567463]]
Gradient check passed!
CHECK GRADIENT
x is 
 [[-0.00116689 -0.00247587  0.0003972  -0.00026163]]
analytic grad is 
 [[ 0.39907169  0.         

In [43]:
X = np.array([[1,-2,3],
              [-1, 2, 0.1]
              ])

Y = np.array([[1,3,3],
              [-1, 2, 0.1]
              ])
print(X*Y)

zer = np.ones((3,4))
print("zer\n",zer)

[[ 1.   -6.    9.  ]
 [ 1.    4.    0.01]]
zer
 [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
